In [1]:
import pandas as pd
import os


# Combine award CSVs

In [2]:
spending_data = "C:/Users/juwon.drake/DOT OST/Volpe-Proj-OS40A323_DOCR - Documents/Task 5E Language Access/LicensingResearch/USA Spending Data"

#_________________________________________________________________________________________________________________________________________________
#_________________________________________________________________________________________________________________________________________________

state_dirs = os.listdir(spending_data)
if 'desktop.ini' in state_dirs:
    state_dirs.pop(state_dirs.index('desktop.ini'))

award_files = []

for dir in state_dirs:
    dir_path = os.path.join(spending_data, dir)
    files = os.listdir(dir_path)
    for file in files:
        if 'Assistance_PrimeAwardSummaries' in file and '.csv' in file:
            award_files.append(os.path.join(dir_path, file))

all_awards = pd.DataFrame()          
try:
    for file in award_files:
        cfile = file
        file_df = pd.read_csv(file, dtype={
            'total_obligated_amount':float,
            'total_outlayed_amount':float,
            'indirect_cost_federal_share_amount':float,
            'total_non_federal_funding_amount':float,
            'total_funding_amount':float,
            'total_face_value_of_loan':float,
            'total_loan_subsidy_cost':float})
        all_awards = pd.concat([all_awards, file_df], ignore_index=True)
except:
    print('Error on {cfile}')

C:\Users\juwon.drake\AppData\Local\Temp\ipykernel_33892\2610031831.py:23: DtypeWarning: Columns (72) have mixed types. Specify dtype option on import or set low_memory=False.
  file_df = pd.read_csv(file, dtype={


# Extract relevant grants

In [3]:
processed_awards = all_awards.copy()

cfda = pd.read_csv("C:\GitHub\LicensingResearch\Data\cfda.csv", dtype={'Number' : str})

processed_awards['cfda_numbers'] = [cfda[0] for cfda in processed_awards.cfda_numbers_and_titles.str.split(':').values]

processed_awards = processed_awards.loc[processed_awards.award_base_action_date_fiscal_year.astype(int) > 2015,].reset_index(drop=True)

cfda_match = []
for x in processed_awards.cfda_numbers:
    if x in cfda.Number.values:
        cfda_match.append(True)
        continue
    cfda_match.append(False)

processed_awards = processed_awards.loc[cfda_match,].reset_index(drop=True)

# Generate report

In [4]:
grouped_awards = processed_awards.groupby(['recipient_state_code', 'awarding_sub_agency_name']).sum()

In [10]:
output = grouped_awards[['total_obligated_amount',
                'total_outlayed_amount',
                'indirect_cost_federal_share_amount',
                'total_non_federal_funding_amount',
                'total_funding_amount',
                'total_face_value_of_loan',
                'total_loan_subsidy_cost']]

for col in output:
    output[col] = output[col].apply(lambda x: "${:.1f}k".format((x/1000)))

output

C:\Users\juwon.drake\AppData\Local\Temp\ipykernel_33892\1689591576.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output[col] = output[col].apply(lambda x: "${:.1f}k".format((x/1000)))


total_obligated_amount  \
recipient_state_code awarding_sub_agency_name                                                
AK                   Federal Motor Carrier Safety Administration                  $1499.9k   
AL                   Federal Motor Carrier Safety Administration                 $15130.4k   
                     National Highway Traffic Safety Administration                $523.5k   
AR                   Federal Motor Carrier Safety Administration                  $4507.6k   
                     National Highway Traffic Safety Administration              $92092.1k   
AS                   National Highway Traffic Safety Administration               $9161.4k   
AZ                   Federal Motor Carrier Safety Administration                  $8371.2k   
                     National Highway Traffic Safety Administration                $721.0k   
CA                   Federal Motor Carrier Safety Administration                    $40.3k   
                     National Highway Traffic Safety Administration                $500.0k   
CO                   Federal Motor Carrier Safety Administration                  $1410.1k   
CT                   Federal Motor Carrier Safety Administration                 $26552.7k   
DC                   Federal Motor Carrier Safety Administration                   $295.9k   
                     National Highway Traffic Safety Administration               $2258.2k   
DE                   Federal Motor Carrier Safety Administration                 $10735.0k   
FL                   Federal Motor Carrier Safety Administration                  $2390.5k   
                     National Highway Traffic Safety Administration               $8956.5k   
GA                   Federal Motor Carrier Safety Administration                  $8602.6k   
GU                   Federal Motor Carrier Safety Administration                    $92.6k   
HI                   Federal Motor Carrier Safety Administration                  $1199.8k   
                     National Highway Traffic Safety Administration              $60523.8k   
IA                   Federal Motor Carrier Safety Administration                 $10074.5k   
                     National Highway Traffic Safety Administration                $575.0k   
ID                   Federal Motor Carrier Safety Administration                  $6584.5k   
                     National Highway Traffic Safety Administration              $42393.5k   
IL                   Federal Motor Carrier Safety Administration                   $-17.5k   
IN                   Federal Motor Carrier Safety Administration                 $12982.9k   
                     National Highway Traffic Safety Administration             $100562.9k   
KY                   National Highway Traffic Safety Administration              $79265.9k   
LA                   Federal Motor Carrier Safety Administration                 $36816.5k   
MA                   Federal Motor Carrier Safety Administration                  $4764.4k   
                     National Highway Traffic Safety Administration                $232.5k   
MD                   Federal Motor Carrier Safety Administration                  $2759.5k   
                     National Highway Traffic Safety Administration              $39344.4k   
MI                   Federal Motor Carrier Safety Administration                  $2617.3k   
MN                   Federal Motor Carrier Safety Administration                  $5550.1k   
                     National Highway Traffic Safety Administration             $124967.6k   
MS                   National Highway Traffic Safety Administration             $115242.0k   
NE                   Federal Motor Carrier Safety Administration                   $959.0k   
NV                   National Highway Traffic Safety Administration               $4297.1k   
OH                   Federal Motor Carrier Safety Administration                  $2331.1k   
                     Natio